In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os.path

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def unpack_pickle(obj_name):
    file_path = "{0}.pkl".format(obj_name)
    n_bytes = 2**31
    max_bytes = 2**31 - 1
    data = bytearray(n_bytes)
    
    ## read
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    loaded_data = pickle.loads(bytes_in)
    return loaded_data

In [3]:
X = unpack_pickle('X')

In [4]:
y = unpack_pickle('y')

In [5]:
X.shape, y.shape

((7730792, 73), (7730792,))

In [6]:
subm = pd.read_csv('../sample_submission.csv')

In [7]:
subm.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.5
1,00000176ac758d54827acd545b6315a5,0.5
2,0000019dcefc128c2d4387c1273dae1d,0.5
3,0000055553dc51b1295785415f1a224d,0.5
4,00000574cefffeca83ec8adf9285b2bf,0.5


In [10]:
test = unpack_pickle('../second_n_third/test')

In [11]:
test.shape, subm.shape

((7853253, 73), (7853253, 2))

In [12]:
X.columns.dtype == test.columns.dtype

True

In [13]:
test.select_dtypes(include='category').columns.tolist()

['Processor',
 'OsPlatformSubRelease',
 'SkuEdition',
 'Census_PrimaryDiskTypeName',
 'Census_PowerPlatformRoleName',
 'Census_OSArchitecture',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'EngineVersion_1',
 'EngineVersion_2',
 'AppVersion_1',
 'AppVersion_2',
 'AvSigVersion_1',
 'Census_OSVersion_1']

In [14]:
dtypes = {}
for i in X.columns:
    dtypes[i] = str(X[i].dtype)

In [15]:
dtypes

{'AVProductStatesIdentifier': 'float32',
 'AVProductsInstalled': 'float16',
 'AppVersion': 'float64',
 'AppVersion_1': 'float64',
 'AppVersion_2': 'float64',
 'AppVersion_3': 'float64',
 'AvSigVersion': 'float64',
 'AvSigVersion_1': 'float64',
 'AvSigVersion_2': 'float64',
 'AvSigVersion_3': 'float64',
 'Census_ActivationChannel': 'float64',
 'Census_ChassisTypeName': 'float64',
 'Census_FirmwareManufacturerIdentifier': 'float16',
 'Census_FirmwareVersionIdentifier': 'float32',
 'Census_FlightRing': 'float64',
 'Census_GenuineStateName': 'float64',
 'Census_HasOpticalDiskDrive': 'int8',
 'Census_InternalBatteryNumberOfCharges': 'float32',
 'Census_InternalBatteryType': 'float64',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches': 'float16',
 'Census_InternalPrimaryDisplayResolutionHorizontal': 'float16',
 'Census_InternalPrimaryDisplayResolutionVertical': 'float16',
 'Census_IsAlwaysOnAlwaysConnectedCapable': 'float16',
 'Census_IsSecureBootEnabled': 'int8',
 'Census_IsTouchEnabled'

In [16]:
for i in dtypes:
    test[i] = test[i].astype(dtypes[i])

In [17]:
test.select_dtypes(include='category').columns.tolist()

[]

In [35]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc
gc.enable()

In [36]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'n_estimators': 4000,
    'colsample_bytree': 0.8, 
    'learning_rate': 0.6000000000000001, 
    'max_depth': 20,
    'metric': 'auc',
    'reg_alpha': 0.6000000000000001, 
    'reg_lambda': 0.5, 
    'subsample': 0.8
}

In [31]:
sub=subm['MachineIdentifier'].to_frame()
sub['HasDetections']=0
sub['1']=0

In [32]:
sub.head()

,MachineIdentifier,HasDetections,1
0,0000010489e3af074adeac69c53e555e,0,0
1,00000176ac758d54827acd545b6315a5,0,0
2,0000019dcefc128c2d4387c1273dae1d,0,0
3,0000055553dc51b1295785415f1a224d,0,0
4,00000574cefffeca83ec8adf9285b2bf,0,0


In [22]:
kfold = 4

In [23]:
X = X.values
y = y.values

In [24]:
test = test.values

In [25]:
gc.collect()

21

In [37]:
skf = StratifiedKFold(n_splits=kfold, random_state=2200)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    dset = lgb.Dataset(X_train, label=y_train)
    dset_eval = lgb.Dataset(X_eval, label=y_eval)
    lgb_model = lgb.train(params, dset, 4000, dset_eval,
                          verbose_eval=100, early_stopping_rounds=100)
    sub['1'] += lgb_model.predict(test, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
     
gc.collect()

 lgb kfold: 1  of  4 : 


/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.726769
[200]	valid_0's auc: 0.730133
[300]	valid_0's auc: 0.731216
[400]	valid_0's auc: 0.731755
[500]	valid_0's auc: 0.731917
[600]	valid_0's auc: 0.731936
[700]	valid_0's auc: 0.731936
Early stopping, best iteration is:
[670]	valid_0's auc: 0.73202
 lgb kfold: 2  of  4 : 
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.726473
[200]	valid_0's auc: 0.729655
[300]	valid_0's auc: 0.730931
[400]	valid_0's auc: 0.731441
[500]	valid_0's auc: 0.731788
Early stopping, best iteration is:
[490]	valid_0's auc: 0.731815
 lgb kfold: 3  of  4 : 
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.726931
[200]	valid_0's auc: 0.730065
[300]	valid_0's auc: 0.731592
[400]	valid_0's auc: 0.73204
[500]	valid_0's auc: 0.732259
[600]	valid_0's auc: 0.732111
Early stopping, best iteration is:
[504]	valid_0's auc: 0.732308
 lgb kfold: 4  of  4 : 
Traini

830

In [38]:
public_nffm_subm = pd.read_csv('nffm_submission.csv')

In [39]:
sub['2'] = public_nffm_subm['HasDetections']

In [40]:
sub['HasDetections'] = (sub['1'] + sub['2']) / 2

In [42]:
sub.head()

,MachineIdentifier,HasDetections,1,2
0,0000010489e3af074adeac69c53e555e,0.579821,0.631393,0.528249
1,00000176ac758d54827acd545b6315a5,0.699366,0.884686,0.514046
2,0000019dcefc128c2d4387c1273dae1d,0.692680,0.835116,0.550244
3,0000055553dc51b1295785415f1a224d,0.492575,0.624165,0.360985
4,00000574cefffeca83ec8adf9285b2bf,0.617335,0.747860,0.486810


In [43]:
sub = sub.drop(columns=['1', '2'])

In [44]:
sub.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.579821
1,00000176ac758d54827acd545b6315a5,0.699366
2,0000019dcefc128c2d4387c1273dae1d,0.692680
3,0000055553dc51b1295785415f1a224d,0.492575
4,00000574cefffeca83ec8adf9285b2bf,0.617335


In [45]:
sub.to_csv('fifth_submit.csv', index=False, float_format='%.5f')

Fifth submit. mean(CV after hyperopt new LGB params + with public NFFM 890 score predict)